In [1]:
import os
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import tarfile
import urllib
import shutil

In [2]:
validation_labels_url = 'https://raw.githubusercontent.com/fh295/semanticCNN/master/imagenet_labels/ILSVRC2012_validation_ground_truth.txt'
validation_mapping_url = 'https://raw.githubusercontent.com/fh295/semanticCNN/master/imagenet_labels/ILSVRC2012_mapping.txt'
root_dir = Path('/workspace/data/imagenet/')
tar_file = list(Path(root_dir).glob('*.tar'))
train_dir = root_dir.joinpath('train')
validation_dir = root_dir.joinpath('validation')

In [4]:
validation_labels = urllib.request.urlopen(validation_labels_url).read().decode().split()
mappings = [i.split(' ') for i in urllib.request.urlopen(validation_mapping_url).read().decode().split('\n')]
mappings = [i for i in mappings if len(i)==2]
mappings = {i[0]: i[1] for i in mappings}
validation_labels = [mappings[i] for i in validation_labels]

In [5]:
validation_dir.mkdir()

In [6]:
validation_tar = tarfile.open(root_dir.joinpath('ILSVRC2012_img_val.tar'))

In [7]:
validation_tar.extractall(validation_dir)

In [8]:
def move_validation_image(image_file):
    index = int(image_file.stem.split('_')[-1])-1
    sysnet = validation_labels[index]
    sysnet_dir = validation_dir.joinpath(sysnet)
    if not sysnet_dir.exists():
        sysnet_dir.mkdir()
    shutil.move(image_file, sysnet_dir.joinpath(image_file.name))

In [9]:
val_images = validation_dir.glob('*.JPEG')

In [10]:
for i in val_images:
    move_validation_image(i)

In [11]:
root_dir.joinpath('ILSVRC2012_img_val.tar').unlink()

In [12]:
train_dir.mkdir()
shutil.move(root_dir.joinpath('ILSVRC2012_img_train.tar'), train_dir.joinpath('ILSVRC2012_img_train.tar'))

PosixPath('/workspace/data/imagenet/train/ILSVRC2012_img_train.tar')

In [13]:
tarfile.open(train_dir.joinpath('ILSVRC2012_img_train.tar')).extractall(train_dir)

In [14]:
def extract_tar(tar_file):
    tarfile.open(tar_file).extractall(train_dir.joinpath(tar_file.stem))
    tar_file.unlink()

In [15]:
pool = ThreadPoolExecutor()

In [16]:
train_dir.joinpath('ILSVRC2012_img_train.tar').unlink()

In [17]:
threads = [pool.submit(extract_tar, tar_file) for tar_file in train_dir.glob('*.tar')]

In [48]:
sum([i.done() for i in threads])

1000